In [14]:
from tensorflow.keras.models import Model
import tensorflow as tf
from model import lstm_unet
import keras.backend as K
from helpers import f1
import warnings
import numpy as np
import matplotlib.pyplot as plt
from data_loader import DataGenerator
warnings.filterwarnings("ignore")

gpus = tf.config.list_physical_devices('GPU')
if gpus:
   tf.config.set_visible_devices(gpus[0], 'GPU')
   tf.config.experimental.set_memory_growth(gpus[0], True)

In [15]:
model = lstm_unet()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 15, 256, 256, 3)]    0         []                            
                                                                                                  
 conv_lstm2d_9 (ConvLSTM2D)  (None, 15, 256, 256, 16)     11008     ['input_2[0][0]']             
                                                                                                  
 batch_normalization_9 (Bat  (None, 15, 256, 256, 16)     64        ['conv_lstm2d_9[0][0]']       
 chNormalization)                                                                                 
                                                                                                  
 time_distributed_8 (TimeDi  (None, 15, 128, 128, 16)     0         ['batch_normalization_9[

In [16]:
train_gen = DataGenerator(
    image_dir="data_v3_processed/train/images/images",
    mask_dir="data_v3_processed/train/masks/masks",
    batch_size=8,
    time_steps=15,
    img_size=(256,256),
    normalize_images=True,
    normalize_masks=False,
    infinite=True,
)

In [17]:
val_gen = DataGenerator(
    image_dir="data_v3_processed/val/images/images",
    mask_dir="data_v3_processed/val/masks/masks",
    batch_size=8,
    time_steps=15,
    img_size=(256,256),
    normalize_images=True,
    normalize_masks=False,
)

In [18]:
history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=1,
                    batch_size=8,
                    steps_per_epoch=2000,
                    validation_steps=13)

2000/2000 [==============================] - 1930s 957ms/step - loss: 0.0601 - f1: 0.8100 - val_loss: 0.0300 - val_f1: 0.0074


In [19]:
model.save("lstm_unet_model.h5")